*1. Introducción *

Low-Rank Adaptation, abreviado como LoRA, es una técnica propuesta en el campo del procesamiento de lenguaje natural y el aprendizaje automático. Su objetivo principal es abordar el desafío de adaptar modelos de lenguaje pre-entrenados, que son muy grandes y tienen una gran cantidad de parámetros, a tareas específicas o dominios sin incurrir en costes muy caros en términos de recursos computacionales y memoria GPU.

La idea central detrás de LoRA es reducir significativamente el número de parámetros entrenables en un modelo al introducir matrices de descomposición de rango entrenables en cada capa de la arquitectura del modelo. Estas matrices de descomposición de rango son matrices más pequeñas que representan de manera eficiente la información de las capas originales del modelo, lo que reduce drásticamente la cantidad de memoria y recursos necesarios para entrenar y utilizar el modelo.

LoRA es una técnica que permite mantener los pesos pre-entrenados de un modelo fijos y reemplazar una parte de los parámetros con matrices de descomposición de rango entrenables. Esto resulta en modelos más eficientes en términos de recursos y memoria, lo que facilita su adaptación a tareas específicas sin comprometer significativamente su rendimiento.

Su base está definida en el paper Hu, Edward, et al. "LORA: Low-Rank Adaptation of Large Language Models." Microsoft Corporation, Version 2, [https://arxiv.org/abs/2106.09685], (2021).

En este trabajo se propone crear un modelo "LoRA" sencillo, para clasificar dígitos utilizando la base de datos MNIST. La idea es "sobrecargar" la red neuronal para que clasifique ineficientemente los dígitos, y, a partir del dígito peor clasificado, hacer fine-tuning utilizando LoRA en este dígito para saber si mejora el resultado de la clasificación.

código adaptado de https://colab.research.google.com/drive/1iERDk94Jp0UErsPf7vXyPKeiM4ZJUQ-a?usp=sharing#scrollTo=WuK0lPwcB7Ia





In [ ]:
#importamos las librerías necesarias
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# hacemos torch determinístico
_ = torch.manual_seed(0)

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# cargamos el dataset MNIST para train
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# creamos un dataloader para train
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

# cargamos MNIST para test
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=10, shuffle=True)

# definimos el device que vamos utilizar
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# creamos una red neuronal demasiado ineficiente para clasificar dígitos MNIST

class desperdicio(nn.Module):
    def __init__(self, hidden_size_1=1000, hidden_size_2=2000):
        super(desperdicio,self).__init__()
        self.linear1 = nn.Linear(28*28, hidden_size_1)
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.linear3 = nn.Linear(hidden_size_2, 10)
        self.relu = nn.ReLU()

    def forward(self, img):
        x = img.view(-1, 28*28)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        return x

net = desperdicio().to(device)

In [ ]:
def train(train_loader, net, epochs=5, total_iterations_limit=None):
    cross_el = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

    total_iterations = 0

    for epoch in range(epochs):
        net.train()

        loss_sum = 0
        num_iterations = 0

        data_iterator = tqdm(train_loader, desc=f'Epoch {epoch+1}')
        if total_iterations_limit is not None:
            data_iterator.total = total_iterations_limit
        for data in data_iterator:
            num_iterations += 1
            total_iterations += 1
            x, y = data
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = net(x.view(-1, 28*28))
            loss = cross_el(output, y)
            loss_sum += loss.item()
            avg_loss = loss_sum / num_iterations
            data_iterator.set_postfix(loss=avg_loss)
            loss.backward()
            optimizer.step()

            if total_iterations_limit is not None and total_iterations >= total_iterations_limit:
                return

train(train_loader, net, epochs=1)

Epoch 1: 100%|██████████| 6000/6000 [07:50<00:00, 12.75it/s, loss=0.236]


In [ ]:
#salvamos los pesos originales para poder recuperarlos
original_weights = {}
for name, param in net.named_parameters():
    original_weights[name] = param.clone().detach()

In [ ]:
#hacemos test en las clasificaciones donde vemos que se equivoca mucho en el dígito 9 en relación a los demás
def test():
    correct = 0
    total = 0

    wrong_counts = [0 for i in range(10)]

    with torch.no_grad():
        for data in tqdm(test_loader, desc='Testing'):
            x, y = data
            x = x.to(device)
            y = y.to(device)
            output = net(x.view(-1, 784))
            for idx, i in enumerate(output):
                if torch.argmax(i) == y[idx]:
                    correct +=1
                else:
                    wrong_counts[y[idx]] +=1
                total +=1
    print(f'Accuracy: {round(correct/total, 3)}')
    for i in range(len(wrong_counts)):
        print(f'wrong counts for the digit {i}: {wrong_counts[i]}')

test()

Testing: 100%|██████████| 1000/1000 [00:05<00:00, 168.34it/s]

Accuracy: 0.954
wrong counts for the digit 0: 31
wrong counts for the digit 1: 17
wrong counts for the digit 2: 46
wrong counts for the digit 3: 74
wrong counts for the digit 4: 29
wrong counts for the digit 5: 7
wrong counts for the digit 6: 36
wrong counts for the digit 7: 80
wrong counts for the digit 8: 25
wrong counts for the digit 9: 116


In [ ]:
# salvamos los parámetros originales
total_parameters_original = 0
for index, layer in enumerate([net.linear1, net.linear2, net.linear3]):
    total_parameters_original += layer.weight.nelement() + layer.bias.nelement()
    print(f'Layer {index+1}: W: {layer.weight.shape} + B: {layer.bias.shape}')
print(f'Total number of parameters: {total_parameters_original:,}')

Layer 1: W: torch.Size([1000, 784]) + B: torch.Size([1000])
Layer 2: W: torch.Size([2000, 1000]) + B: torch.Size([2000])
Layer 3: W: torch.Size([10, 2000]) + B: torch.Size([10])
Total number of parameters: 2,807,010


La idea básica detrás de LoRA es mantener las matrices pre-entrenadas (es decir, los parámetros del modelo original) congeladas (en un estado fijo) y solo agregar un pequeño delta a la matriz original, que tiene menos parámetros que la matriz original.

Por ejemplo, consideremos la matriz `W`, que podría ser los parámetros de una capa completamente conectada o una de las matrices del mecanismo de autoatención de un transformer:

W_orig = W + DeltaW

Si `W_orig` tuviera dimensiones `n x m` y simplemente inicializáramos una nueva matriz delta con las mismas dimensiones para afinarla, no habríamos ganado nada; más bien al contrario, habríamos duplicado los parámetros.

El truco consiste en hacer que `DeltaW` sea menos "dimensional" que la matriz original, construyéndola mediante una multiplicación de matrices a partir de matrices de dimensiones más bajas `B` y `A`:

DeltaW = B * A


Primero definimos un rango `r`, que es significativamente menor que las dimensiones básicas de la matriz, `r << n` y `r << m`. La matriz `B` es de dimensiones `n x r` y la matriz `A` es de dimensiones `r x m`. Multiplicarlas produce una matriz con las mismas dimensiones que `W`, pero construida a partir de una cantidad mucho menor de parámetros.

Queremos que nuestro delta sea cero al comienzo del entrenamiento, de modo que el fine tuning comience de la misma manera que el modelo original. Por lo tanto, `B` se inicializa como todo ceros y `A` se inicializa como valores aleatorios (generalmente distribuidos de manera normal).

Además, en el paper de LoRA,la matriz delta está definida desde un  parámetro `alpha`:

DeltaW = alpha * B * A

Si configuramos el `alpha` con el primer `r` con el que intentamos y ajustamos la tasa de aprendizaje, podemos cambiar el parámetro `r` más tarde sin tener que volver a ajustar la tasa de aprendizaje (learning rate).

En este sentido, la implementación de LoRA se realiza mediante la aplicación de una serie de transformaciones a las matrices de peso de una red neuronal. Estas transformaciones están diseñadas para preservar la información importante en los pesos mientras reducen su dimensionalidad. Las técnicas principales utilizadas en LoRA son:

Transformación Lineal: Se aplica una transformación lineal a las matrices de peso para transformarlas en un espacio de menor dimensión.

Aproximación de la Matriz de Peso: Las matrices de peso se aproximan utilizando una técnica de aproximación dispersa.

Descomposición en Valores Singulares (SVD): La SVD se utiliza para factorizar las matrices de peso en el producto de tres matrices. Esta descomposición permite extraer los valores y vectores singulares más importantes, lo que conduce a una representación de bajo rango de los pesos.

En LoRA, la SVD es utilizada para identificar y retener solo los componentes más significativos de las matrices de peso. Al factorizar una matriz de peso y conservar solo los valores singulares más grandes (y sus vectores correspondientes), se puede lograr una representación de bajo rango de esa matriz. Esto es particularmente útil para simplificar modelos de red neuronal para su despliegue en dispositivos con recursos limitados.

Más especificamente, LoRA es una técnica diseñada para modificar modelos de redes neuronales pre-entrenados sin tener que reentrenar todo el modelo. Funciona introduciendo módulos "aprendibles", que pueden considerarse como "bloques LoRA", en el modelo. Estos bloques LoRA aplican esencialmente una forma truncada de la Descomposición en Valores Singulares (SVD). En otras palabras, LoRA mantiene los pesos principales del modelo pre-entrenado congelados y solo introduce algunos módulos de SVD truncados (los bloques LoRA) en el modelo.

In [ ]:
class LoRAParametrization(nn.Module):
    def __init__(self, features_in, features_out, rank=1, alpha=1, device='cpu'):
        super().__init__()
        # nos basamos en la sección 4.1 del paper:
        #  Usamos una inicalización gaussiana para A y cero para B, donde ∆W = BA es cero al comienzo del entreinamiento
        self.lora_A = nn.Parameter(torch.zeros((rank,features_out)).to(device))
        self.lora_B = nn.Parameter(torch.zeros((features_in, rank)).to(device))
        nn.init.normal_(self.lora_A, mean=0, std=1)

        # en la misma sección
        #   se escala ∆Wx por α/r , donde α es una constante en r.
        #   cuando optimizamos usando Adam, hacer tuning a α es practicamente lo mismo que hacer tuning al learning rate si escalamos  la inicialización.
        #   Entonces ponemos α al primer r que encontramos y no tuneamos.
        #   La escala ayuda a reducir la necesiadd de retunear los hyperparametros cuando variamos r.
        self.scale = alpha / rank
        self.enabled = True

    def forward(self, original_weights):
        if self.enabled:
            # Return W + (B*A)*scale
            return original_weights + torch.matmul(self.lora_B, self.lora_A).view(original_weights.shape) * self.scale
        else:
            return original_weights

In [ ]:
import torch.nn.utils.parametrize as parametrize

def linear_layer_parameterization(layer, device, rank=1, lora_alpha=1):
    # solo añadimos la parametrización a la matriz de pesos, ignoramos el sesgo

    # cogemos la sección 4.2 del paper:
    #  Solo vamos estudiar los attention weights

    features_in, features_out = layer.weight.shape
    return LoRAParametrization(
        features_in, features_out, rank=rank, alpha=lora_alpha, device=device
    )

parametrize.register_parametrization(
    net.linear1, "weight", linear_layer_parameterization(net.linear1, device)
)
parametrize.register_parametrization(
    net.linear2, "weight", linear_layer_parameterization(net.linear2, device)
)
parametrize.register_parametrization(
    net.linear3, "weight", linear_layer_parameterization(net.linear3, device)
)


def enable_disable_lora(enabled=True):
    for layer in [net.linear1, net.linear2, net.linear3]:
        layer.parametrizations["weight"][0].enabled = enabled

In [ ]:
total_parameters_lora = 0
total_parameters_non_lora = 0
for index, layer in enumerate([net.linear1, net.linear2, net.linear3]):
    total_parameters_lora += layer.parametrizations["weight"][0].lora_A.nelement() + layer.parametrizations["weight"][0].lora_B.nelement()
    total_parameters_non_lora += layer.weight.nelement() + layer.bias.nelement()
    print(
        f'Layer {index+1}: W: {layer.weight.shape} + B: {layer.bias.shape} + Lora_A: {layer.parametrizations["weight"][0].lora_A.shape} + Lora_B: {layer.parametrizations["weight"][0].lora_B.shape}'
    )
# Los parámetros non-LoRA tienen que matchear con los originales
assert total_parameters_non_lora == total_parameters_original
print(f'Total number of parameters (original): {total_parameters_non_lora:,}')
print(f'Total number of parameters (original + LoRA): {total_parameters_lora + total_parameters_non_lora:,}')
print(f'Parameters introduced by LoRA: {total_parameters_lora:,}')
parameters_incremment = (total_parameters_lora / total_parameters_non_lora) * 100
print(f'Parameters incremment: {parameters_incremment:.3f}%')

Layer 1: W: torch.Size([1000, 784]) + B: torch.Size([1000]) + Lora_A: torch.Size([1, 784]) + Lora_B: torch.Size([1000, 1])
Layer 2: W: torch.Size([2000, 1000]) + B: torch.Size([2000]) + Lora_A: torch.Size([1, 1000]) + Lora_B: torch.Size([2000, 1])
Layer 3: W: torch.Size([10, 2000]) + B: torch.Size([10]) + Lora_A: torch.Size([1, 2000]) + Lora_B: torch.Size([10, 1])
Total number of parameters (original): 2,807,010
Total number of parameters (original + LoRA): 2,813,804
Parameters introduced by LoRA: 6,794
Parameters incremment: 0.242%


In [ ]:
# congelamos los parámetros no-Lora
for name, param in net.named_parameters():
    if 'lora' not in name:
        print(f'Freezing non-LoRA parameter {name}')
        param.requires_grad = False

# cargamos el mnist otra vez pero solo con el 9 (fine-tuning)
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
exclude_indices = mnist_trainset.targets == 9
mnist_trainset.data = mnist_trainset.data[exclude_indices]
mnist_trainset.targets = mnist_trainset.targets[exclude_indices]
# creamos un dataloader para el training
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

# entrenamos la red con LoRA solo en el dígito 9 y solo en 100 batches
train(train_loader, net, epochs=1, total_iterations_limit=100)

Freezing non-LoRA parameter linear1.bias
Freezing non-LoRA parameter linear1.parametrizations.weight.original
Freezing non-LoRA parameter linear2.bias
Freezing non-LoRA parameter linear2.parametrizations.weight.original
Freezing non-LoRA parameter linear3.bias
Freezing non-LoRA parameter linear3.parametrizations.weight.original


Epoch 1:  99%|█████████▉| 99/100 [00:03<00:00, 24.77it/s, loss=0.102]


In [ ]:
# chequeamos si los parámetros originales no han sido modificados por el fine-tuning
assert torch.all(net.linear1.parametrizations.weight.original == original_weights['linear1.weight'])
assert torch.all(net.linear2.parametrizations.weight.original == original_weights['linear2.weight'])
assert torch.all(net.linear3.parametrizations.weight.original == original_weights['linear3.weight'])

enable_disable_lora(enabled=True)
# el nuevo peso linear1.weight se obtiene a través de la función "forward" de la parametrizacion LoRA
# los pesos originales se mueven a net.linear1.parametrizations.weight.original
# información de aquí: https://pytorch.org/tutorials/intermediate/parametrizations.html#inspecting-a-parametrized-module
assert torch.equal(net.linear1.weight, net.linear1.parametrizations.weight.original + (net.linear1.parametrizations.weight[0].lora_B @ net.linear1.parametrizations.weight[0].lora_A) * net.linear1.parametrizations.weight[0].scale)

enable_disable_lora(enabled=False)
# si quitamos el LoRa, el linear1.weight es el original
assert torch.equal(net.linear1.weight, original_weights['linear1.weight'])

Resultados:

podemos ver que aplicando LoRA mejoramos considerablemente la clasificación. El hiperparámetro  r juega un papel crucial en LoRA, determinando el rango de las matrices de baja dimensionalidad utilizadas para la adaptación. Un valor de
r más pequeño simplifica la matriz de baja dimensionalidad, acelerando el entrenamiento, pero puede reducir la calidad de la adaptación y aumentar el riesgo de un ajuste insuficiente. Por otro lado, un valor de  r más alto aumenta la complejidad pero mejora la capacidad del modelo para capturar información específica de la tarea. Encontrar el equilibrio adecuado experimentando con diferentes valores de  r es fundamental para lograr un rendimiento óptimo en nuevas tareas.

In [ ]:
# Testeamos con LoRa
enable_disable_lora(enabled=True)
test()

Testing: 100%|██████████| 1000/1000 [00:14<00:00, 68.88it/s]

Accuracy: 0.924
wrong counts for the digit 0: 47
wrong counts for the digit 1: 27
wrong counts for the digit 2: 65
wrong counts for the digit 3: 240
wrong counts for the digit 4: 89
wrong counts for the digit 5: 32
wrong counts for the digit 6: 54
wrong counts for the digit 7: 137
wrong counts for the digit 8: 61
wrong counts for the digit 9: 9


In [ ]:
# Testeamos sin LoRa
#vemos una mejora considerable con Lora
enable_disable_lora(enabled=False)
test()

Testing: 100%|██████████| 1000/1000 [00:06<00:00, 159.45it/s]

Accuracy: 0.954
wrong counts for the digit 0: 31
wrong counts for the digit 1: 17
wrong counts for the digit 2: 46
wrong counts for the digit 3: 74
wrong counts for the digit 4: 29
wrong counts for the digit 5: 7
wrong counts for the digit 6: 36
wrong counts for the digit 7: 80
wrong counts for the digit 8: 25
wrong counts for the digit 9: 116
